# Notebook 4: Transcription factor binding sites in *Escherichia coli*

### by Justin B. Kinney

In this example, we will practice analyzing a small biological dataset. Specifically, we will statistically characterize the DNA sequences of a transcription factor, CRP,  in the bacterium  *Escherichia coli*. CRP is one of the most pleiotropic *E. coli* transcriptio factor, with over 350 functional binding sites throughout the bacterium's 4.6 Mb genome.  

In [ ]:
# Always put this first
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

If you open 'binding_site_db.txt', you will see that TF names are listed in the second column. First we need to know what the delimeter is. Let's load all lines and look at the last one. 

In [ ]:
# Open binding site file for viewing (might not work on Windows)
!open binding_site_db.txt

In [ ]:
# Open file in Python and load all lines into list
db_local_file = "binding_site_db.txt"
f = open(db_local_file)
lines = f.readlines()
print(len(lines))

In [ ]:
# Look at what the first line is
print('First line:\n', repr(lines[0]))

In [ ]:
# Look at what the last line is
print('Last line:\n', repr(lines[-1]))

Ah, the file uses tabs. So the quickest thing that might work is to keep only lines that contain "CRP" with a tab character on either side. Let's try that

In [ ]:
# Get lines for CRP
string_to_match = '\tCRP\t'
lines_we_want = [l for l in lines if string_to_match in l]
len(lines_we_want)

Now we have a list of lines. Here are the first and last lines.

In [ ]:
# Check first line and the last line
print("First line:\n", lines_we_want[0])
print("Last line:\n", lines_we_want[-1])

In [ ]:
# Split lines into atoms
x = lines_we_want[-1]
atoms = x.split('\t')
print(atoms)

In [ ]:
# Extract DNA sequence
print(atoms[11].upper())

In [ ]:
# Extract all sites
sites = [x.split('\t')[11].upper() for x in lines_we_want]
print(sites)

In [ ]:
# Let's figure out how many sites there are of each length
length_dict = {}
for site in sites:
    length = len(site)
    if length in length_dict.keys():
        length_dict[length] += 1
    else:
        length_dict[length] = 1
print(length_dict)

In [ ]:
# Now find the length with the most sites
counts = list(length_dict.values())
lengths = list(length_dict.keys())    
index = np.argmax(counts)
chosen_length = lengths[index]
chosen_counts = counts[index]
print('Choosing length %d: %d sites'%(chosen_length,chosen_counts))

In [ ]:
# Grab the sites of length 42
chosen_sites = [site for site in sites if len(site)==chosen_length]
print(len(chosen_sites))
print(chosen_sites[:5])

Now let's count how many times each base occurs at each position in this list. After a little thought we conclude that we want to do something like this:

In [ ]:
# Fill in counts matrix
counts_matrix = np.zeros([chosen_length,4])
bases = 'ACGT'
for s in chosen_sites:
    for i in range(chosen_length):
        for b, base in enumerate(bases):
            counts_matrix[i,b] += (s[i] == base)
print(counts_matrix)

In [ ]:
# Turn this into a function that can process any TF name!
def get_counts_matrix(tf_name):
    print('Getting counts for %s...'%tf_name)
    
    # Get lines for tf
    string_to_match = '\t%s\t'%tf_name
    lines_we_want = [l for l in lines if string_to_match in l]
    assert len(lines_we_want) > 0, \
        'No lines found for TF %s'%tf_name
        
    # Parse sites from lines
    sites = [x.split('\t')[11].upper() for x in lines_we_want]

    # Figure out how many sites there are of each length
    length_dict = {}
    for site in sites:
        l = len(site)
        if l in length_dict.keys():
            length_dict[l] += 1
        else:
            length_dict[l] = 1
    print('length_dict: %s'%str(length_dict))
            
    # Now find the length with the most sites
    counts = list(length_dict.values())
    lengths = list(length_dict.keys())   
    index = np.argmax(counts)
    chosen_length = lengths[index]
    chosen_counts = counts[index]
    print('Choosing length %d: %d sites'%(chosen_length,chosen_counts))
    
    # Get sites of chosen length
    chosen_sites = [site for site in sites if len(site)==chosen_length]

    counts_matrix = np.zeros([chosen_length,4])
    bases = 'ACGT'
    for s in chosen_sites:
        for i in range(chosen_length):
            for b, base in enumerate(bases):
                counts_matrix[i,b] += (s[i] == base)
                
    return counts_matrix

In [ ]:
get_counts_matrix('CRP')

Looks reasonable, but it would be nice to have a graphical representation of this.

In [ ]:
counts_matrix = get_counts_matrix('CRP')
plt.imshow(counts_matrix)
plt.show()

Hard to see. Flip this thing on it's side and make it bigger. Also, which colors mean what?

In [ ]:
plt.figure(figsize=[12,2])
plt.imshow(counts_matrix.T)
plt.colorbar()
plt.show()

That's better, but it will still take some playing around with to make it pretty. It's all blurry, the y-axis labels are meaningless, etc.

Just have to play around a while until you get something that looks presentable. Here's what I came up with. 

In [ ]:
# Compute occurence frequency of each base at each position, not the total counts
num_sites = len(sites)
freq_matrix = counts_matrix.T/num_sites
L = counts_matrix.shape[0]

# Set plotting parameter
figure_size = [12,3]
label_size = 16
title_size = 24
colormap = plt.get_cmap('Greens')

# Specify figure of proper size
plt.figure(figsize=figure_size)

# Show matrix without any smoothing
plt.imshow(freq_matrix, cmap=colormap)

# Put interpretable letters on y-axis
plt.yticks(range(4),['A','C','G','T'], fontsize=label_size)

# Label positions symmetically
positions = np.arange(L)-(L/2)+1
indices = np.arange(0,L,5)
plt.xticks(indices+.5, positions[indices].astype(int), fontsize=label_size)

# Fix colorbar
plt.clim([0, 1])
cbar = plt.colorbar(ticks=np.linspace(0,1,5))
cbar.ax.tick_params(labelsize=label_size) 

# Draw a title
plt.title('CRP base frequency matrix', fontsize=title_size)

# Fix spacing in plot
plt.tight_layout()

# Save the figure
picture_file = 'crp_matrix.png'
plt.savefig(picture_file)

# Draw the plot
plt.show()



Let's check to make sure this actually worked.

In [ ]:
# WARNING: Might not work in Windows
!open $picture_file

We should also save a text file that has the base counts at each position, so we can remake plots like this whenever we want

In [ ]:
# Save counts to text file
counts_matrix_file = 'crp_counts_matrix.txt'
np.savetxt(counts_matrix_file,counts_matrix)

In [ ]:
# WARNING: Might not work in Windows
!open $counts_matrix_file

This is hard for us humans to read. Let's change the text format so that we can eyeball this when we need to

In [ ]:
# Save counts to text file more cleanly
np.savetxt(counts_matrix_file,counts_matrix, fmt='%d', delimiter='\t')

In [ ]:
# WARNING: Might not work in Windows
!open $counts_matrix_file

We're done!